In [2]:
import pandas as pd
import ssl
import numpy as np
from pathlib import Path
import sqlalchemy
import datetime

# from utils.helpers import alpaca_symbol_data

# For creating the DB
database_connection_string = 'sqlite:///SP500.db'
engine = sqlalchemy.create_engine(database_connection_string)

# prevent an ssl failure
ssl._create_default_https_context = ssl._create_unverified_context

payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

# There are 2 tables on the Wikipedia page
# we want the first table
first_table = payload[0]
second_table = payload[1]

df500table = first_table

sp500Tickers = df500table['Symbol'].values.tolist()
df500sectors = df500table.drop(columns={'Security','SEC filings','GICS Sub-Industry', 'Headquarters Location', 'Date first added', 'CIK','Founded'})
df500sectors.set_index('Symbol', inplace=True)
# print(sp500Tickers)

In [3]:
StocksWide = pd.read_sql_table('StocksWide', engine)
StocksWide.drop(columns='index', inplace=True)
StocksWide = StocksWide.set_index('timestamp')
StocksWide

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-04-13 04:00:00,0.010410,0.016216,-0.007205,0.015125,0.001759,0.001968,-0.011522,0.016192,-0.004815,0.015665,...,0.011212,0.013618,0.008143,0.002355,-0.010348,0.008414,0.005851,-0.010332,-0.007684,0.007830
2021-04-14 04:00:00,-0.008490,-0.025272,0.001600,-0.022000,-0.003013,0.009064,0.002824,-0.006947,0.000844,-0.006556,...,-0.007537,0.004826,0.017936,0.018744,0.007477,0.005360,-0.003552,-0.005917,0.015529,-0.005670
2021-04-15 04:00:00,0.004598,-0.021653,0.006214,0.004781,0.008491,0.005066,0.012594,0.013658,-0.003878,0.014951,...,-0.014030,0.020566,-0.004716,-0.005371,0.001019,-0.007259,0.018743,0.005353,-0.011160,0.009272
2021-04-16 04:00:00,0.003954,-0.003169,0.008574,-0.003308,0.003907,0.001590,-0.026277,-0.000322,0.006312,0.002310,...,-0.005785,0.001423,-0.010651,-0.000306,-0.005585,-0.004889,-0.004979,0.004952,-0.004079,0.008179
2021-04-19 04:00:00,-0.006549,-0.019563,-0.005424,0.010110,0.005928,-0.004167,0.003415,0.007524,-0.001951,-0.013419,...,-0.002561,-0.000567,-0.005634,0.000611,-0.006974,-0.005660,0.003548,-0.007537,-0.004802,0.003582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-07 04:00:00,0.022799,-0.009351,0.028212,0.006196,0.033459,0.030292,0.010438,0.031522,0.008201,0.020789,...,-0.025942,-0.002275,0.014916,0.000208,0.003820,0.018010,0.020342,0.000765,-0.006135,0.035199
2022-04-08 04:00:00,-0.006190,-0.016598,0.037742,-0.009665,0.011329,0.011742,-0.002632,-0.001377,-0.010250,-0.009255,...,-0.017813,0.000936,0.016624,0.000625,-0.003106,0.003790,-0.002386,-0.001513,0.003298,0.011628
2022-04-11 04:00:00,-0.014713,0.028485,-0.021131,-0.017894,-0.029543,-0.018407,-0.042728,-0.027859,-0.026272,-0.009959,...,-0.000140,-0.007077,-0.023524,0.026032,-0.009245,-0.005213,-0.017041,-0.005064,0.004221,-0.032517


In [4]:
sectors = df500sectors['GICS Sector'].unique()
sectors

array(['Industrials', 'Health Care', 'Information Technology',
       'Communication Services', 'Consumer Staples',
       'Consumer Discretionary', 'Utilities', 'Financials', 'Materials',
       'Real Estate', 'Energy'], dtype=object)

In [5]:
#Looking at relative sizes of Sectors
df500sectors['GICS Sector'].value_counts()

Information Technology    74
Industrials               73
Financials                66
Health Care               65
Consumer Discretionary    60
Consumer Staples          32
Real Estate               30
Utilities                 29
Materials                 28
Communication Services    26
Energy                    21
Name: GICS Sector, dtype: int64

In [6]:
SectorDF = StocksWide[['SPY']]
SectorDF

,SPY
timestamp,
2021-04-13 04:00:00,0.003134
2021-04-14 04:00:00,-0.003415
2021-04-15 04:00:00,0.005027
2021-04-16 04:00:00,-0.000072
2021-04-19 04:00:00,-0.002522
...,...
2022-04-07 04:00:00,0.006956
2022-04-08 04:00:00,-0.001049
2022-04-11 04:00:00,-0.009435


In [7]:
# Iterates through StocksWide and puts all the stocks into a sector before taking a mean

stockstodrop = []
StocksWideNoSPY = StocksWide.drop(columns='SPY')
for sector in sectors:
    for (columnName, columnValues) in StocksWideNoSPY.iteritems():
        if sector != df500sectors.loc[columnName]['GICS Sector']:
            stockstodrop.append(columnName)
    TempDF = StocksWideNoSPY.drop(columns=stockstodrop)
    stockstodrop = []
    SectorDF[sector] = TempDF.mean(axis=1)


C:\Users\canav\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [8]:
SectorDF

,SPY,Industrials,Health Care,Information Technology,Communication Services,Consumer Staples,Consumer Discretionary,Utilities,Financials,Materials,Real Estate,Energy
timestamp,,,,,,,,,,,,
2021-04-13 04:00:00,0.003134,-0.001641,0.006510,0.000045,-0.000767,0.000146,-0.000002,0.016096,-0.004232,-0.003210,0.007684,0.001140
2021-04-14 04:00:00,-0.003415,-0.000804,0.000509,-0.004617,-0.004491,-0.001597,-0.002539,0.007733,0.010694,0.009282,-0.004979,0.020342
2021-04-15 04:00:00,0.005027,0.000453,0.011926,0.004855,0.003417,0.005316,-0.004535,0.011355,-0.001082,0.004324,0.011913,-0.006888
2021-04-16 04:00:00,-0.000072,-0.003498,0.002819,-0.002692,-0.003881,0.001754,0.002421,0.002735,-0.004246,-0.001499,-0.003487,-0.018957
2021-04-19 04:00:00,-0.002522,-0.004149,-0.000356,-0.009452,-0.002727,-0.002429,-0.007847,-0.005995,-0.003404,-0.005562,0.002253,-0.001944
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-07 04:00:00,0.006956,0.005856,0.021049,0.005208,-0.009337,0.008555,0.004695,-0.003217,-0.001480,0.004416,-0.004456,0.002626
2022-04-08 04:00:00,-0.001049,-0.005791,0.005426,-0.007141,0.001863,0.001890,0.004482,0.001318,0.002899,0.002020,0.001696,0.027976
2022-04-11 04:00:00,-0.009435,-0.001636,-0.016449,-0.008222,-0.001345,-0.002333,0.006947,-0.014041,-0.002284,-0.001462,-0.010802,-0.018648


In [9]:
(SectorDF+1).cumprod()

,SPY,Industrials,Health Care,Information Technology,Communication Services,Consumer Staples,Consumer Discretionary,Utilities,Financials,Materials,Real Estate,Energy
timestamp,,,,,,,,,,,,
2021-04-13 04:00:00,1.003134,0.998359,1.006510,1.000045,0.999233,1.000146,0.999998,1.016096,0.995768,0.996790,1.007684,1.001140
2021-04-14 04:00:00,0.999708,0.997556,1.007023,0.995428,0.994745,0.998548,0.997459,1.023953,1.006417,1.006042,1.002667,1.021505
2021-04-15 04:00:00,1.004734,0.998008,1.019033,1.000262,0.998143,1.003856,0.992936,1.035580,1.005328,1.010392,1.014612,1.014469
2021-04-16 04:00:00,1.004661,0.994517,1.021905,0.997569,0.994270,1.005617,0.995339,1.038413,1.001059,1.008877,1.011074,0.995238
2021-04-19 04:00:00,1.002127,0.990391,1.021541,0.988140,0.991558,1.003175,0.987529,1.032188,0.997652,1.003266,1.013351,0.993304
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-07 04:00:00,1.032680,0.886488,1.145055,0.961191,0.855463,1.051411,0.835105,1.109873,0.921410,0.994521,1.070898,1.337260
2022-04-08 04:00:00,1.031597,0.881354,1.151267,0.954327,0.857057,1.053398,0.838848,1.111336,0.924081,0.996530,1.072714,1.374671
2022-04-11 04:00:00,1.021864,0.879913,1.132330,0.946481,0.855904,1.050941,0.844675,1.095732,0.921970,0.995073,1.061126,1.349036


In [10]:
SectorDF.to_sql('SectorDF', engine, if_exists='replace')

# Market Cap Sleeves

Work in process, do not reference past this

In [12]:
SP500_Constituents_details = pd.read_csv(Path("resources/constituents_financials.csv"), index_col='Symbol')
SP500_Constituents_details = SP500_Constituents_details.sort_values(by=['Market Cap'], ascending=False)
SP500_Constituents_details.head()

,Name,Sector,Price,Price/Earnings,Dividend Yield,Earnings/Share,52 Week Low,52 Week High,Market Cap,EBITDA,Price/Sales,Price/Book,SEC Filings
Symbol,,,,,,,,,,,,,
AAPL,Apple Inc.,Information Technology,155.15,16.86,1.579541,9.20,180.10,131.1200,8.095080e+11,7.938600e+10,3.458609,5.66,http://www.sec.gov/cgi-bin/browse-edgar?action...
GOOGL,Alphabet Inc Class A,Information Technology,1007.71,31.48,0.000000,22.27,1198.00,824.3000,7.338240e+11,3.421700e+10,6.801692,4.70,http://www.sec.gov/cgi-bin/browse-edgar?action...
GOOG,Alphabet Inc Class C,Information Technology,1001.52,40.29,0.000000,22.27,1186.89,803.1903,7.285360e+11,3.271400e+10,6.772653,4.67,http://www.sec.gov/cgi-bin/browse-edgar?action...
MSFT,Microsoft Corp.,Information Technology,85.01,25.76,1.874791,2.97,96.07,63.2200,6.899780e+11,4.107900e+10,7.113097,9.49,http://www.sec.gov/cgi-bin/browse-edgar?action...
AMZN,Amazon.com Inc,Consumer Discretionary,1350.50,296.16,0.000000,6.16,1498.00,812.5000,6.858730e+11,1.613200e+10,3.927053,24.28,http://www.sec.gov/cgi-bin/browse-edgar?action...


In [13]:
StockCount = 0
Sleeve = 1
StockMarketCap = [[],[],[],[],[],[]]

for index, row in SP500_Constituents_details.iterrows():
    if StockCount == Sleeve * 100:
        Sleeve += 1
    StockMarketCap[Sleeve-1].append(index)
    StockCount += 1

In [14]:
StocksWide.head()

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-04-13 04:00:00,0.010410,0.016216,-0.007205,0.015125,0.001759,0.001968,-0.011522,0.016192,-0.004815,0.015665,...,0.011212,0.013618,0.008143,0.002355,-0.010348,0.008414,0.005851,-0.010332,-0.007684,0.007830
2021-04-14 04:00:00,-0.008490,-0.025272,0.001600,-0.022000,-0.003013,0.009064,0.002824,-0.006947,0.000844,-0.006556,...,-0.007537,0.004826,0.017936,0.018744,0.007477,0.005360,-0.003552,-0.005917,0.015529,-0.005670
2021-04-15 04:00:00,0.004598,-0.021653,0.006214,0.004781,0.008491,0.005066,0.012594,0.013658,-0.003878,0.014951,...,-0.014030,0.020566,-0.004716,-0.005371,0.001019,-0.007259,0.018743,0.005353,-0.011160,0.009272
2021-04-16 04:00:00,0.003954,-0.003169,0.008574,-0.003308,0.003907,0.001590,-0.026277,-0.000322,0.006312,0.002310,...,-0.005785,0.001423,-0.010651,-0.000306,-0.005585,-0.004889,-0.004979,0.004952,-0.004079,0.008179
2021-04-19 04:00:00,-0.006549,-0.019563,-0.005424,0.010110,0.005928,-0.004167,0.003415,0.007524,-0.001951,-0.013419,...,-0.002561,-0.000567,-0.005634,0.000611,-0.006974,-0.005660,0.003548,-0.007537,-0.004802,0.003582


In [15]:
SleeveDf = StocksWide[['SPY']]
SleeveDf

,SPY
timestamp,
2021-04-13 04:00:00,0.003134
2021-04-14 04:00:00,-0.003415
2021-04-15 04:00:00,0.005027
2021-04-16 04:00:00,-0.000072
2021-04-19 04:00:00,-0.002522
...,...
2022-04-07 04:00:00,0.006956
2022-04-08 04:00:00,-0.001049
2022-04-11 04:00:00,-0.009435


In [16]:
SleeveDf = StocksWide[['SPY']]

for sleeve in StockMarketCap:
    for stock in sleeve:
        TempDF
        